# What is an API?
Let's start with the APIs. The first thing you need to know is: **APIs are everywhere!**

- API is short for Application Programming Interface. 
- APIs allow two computers to communicate with each other and exchange information. They are used widely in data science and we will see some use-cases during today's exercise.

<img src="data/api.png">

# Examples of API
- Weather API ([example](https://openweathermap.org/api))
- PayPal API ([example](https://developer.paypal.com/docs/api/overview/))
- Google Sheets API ([example](https://developers.google.com/sheets/api))
- Google Ads API ([example](https://developers.google.com/google-ads/api/docs/start))
- Facebook Marketin API ([example](https://developers.facebook.com/docs/marketing-apis/))

# What is HTTP?
- HTTP stands for Hypertext Transfer Protocol and is used to structure requests and responses over the internet. HTTP requires data to be transferred from one point to another over the network.

## HTTP request methods:
- GET
- POST
- PUT
- DELETE

In [ ]:
# import library
import requests
import pandas as pd

In [ ]:
# load google search page with GET method
r = requests.get('https://www.google.sk/')
r

## HTTP response status codes
HTTP response status codes indicate whether a specific HTTP request has been successfully completed. Responses are grouped in five classes:

- Informational responses (100–199)
- Successful responses (200–299)
- Redirects (300–399)
- Client errors (400–499)
- Server errors (500–599)

All status codes can be found [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

In [ ]:
# chceck status code
r.status_code

In [ ]:
# chceck the headers
r.headers

In [ ]:
# url of request
r.url

In [ ]:
# encoding
r.encoding

In [ ]:
# html content
r.text

In [ ]:
# create params
params = {
    'key1': 'value1', 
    'key2': 'value2'
}

r = requests.get('https://httpbin.org/get', params=params)

In [ ]:
# check the url
r.url

In [ ]:
# json
r.json()

In [ ]:
# fake User-Agent 
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
}

params = {
    'key1': 'value1', 
    'key2': 'value2'
}

r = requests.get('https://httpbin.org/get', params=params, headers=headers)

In [ ]:
r.json()

In [ ]:
# POST METHOD
data = {
    'name': 'name_here',
    'surname': 'surname_here'
}
r = requests.post('https://httpbin.org/post', json = data)
r.json()

# Own API with Flask

In [ ]:
##################################
# DO NOT RUN THIS CELL IN JUPYTER
#################################
from flask import Flask
from flask_restful import Resource, Api, reqparse

# create app
app = Flask(__name__)

# create api
api = Api(app)

items = []

# create item endpoint
class Item(Resource):
    parser = reqparse.RequestParser()
    parser.add_argument('price',
        type=float,
        required=True,
        help="This field cannot be left blank!"
    )

    def get(self, name):
        matched_items = [item for item in items if item['name'] == name]
        if not matched_items:
            return {'item': None}
        
        return {'item': matched_items[0]}

    def post(self, name):
        matched_items = [item for item in items if item['name'] == name]
        
        if matched_items:
            return {'message': f"An item with name '{name}' already exists."}

        data = Item.parser.parse_args()

        item = {'name': name, 'price': data['price']}
        items.append(item)
        
        return {
            'message': 'Item created',
            'item': item
        }

    def delete(self, name):
        global items
        items = [item for item in items if item['name'] != name]
        
        return {'message': 'Item deleted'}

    def put(self, name):
        data = Item.parser.parse_args()
        
        matched_items = [item for item in items if item['name'] == name]
        if not matched_items:
            item = {'name': name, 'price': data['price']}
            items.append(item)
        else:
            matched_items[0].update(data)
        
        return {'message': 'Item updated'}

# create items endpoinnt
class ItemList(Resource):
    def get(self):
        return {'items': items}

# register endpoints
api.add_resource(Item, '/item/<string:name>')
api.add_resource(ItemList, '/items')

# if __name__ == '__main__':
#     app.run(debug=True)

In [ ]:
# check the items
r = requests.get('http://127.0.0.1:5000/items')
r.json()

In [ ]:
# create an item without price (this will cause an error)
r = requests.post('http://127.0.0.1:5000/item/desk')
r.json()

In [ ]:
# create an item with price
data = {
    'price': 12
}
r = requests.post('http://127.0.0.1:5000/item/desk', data=data)
r.json()

In [ ]:
# check the items
r = requests.get('http://127.0.0.1:5000/items')
r.json()

In [ ]:
data = {
    'price': 20
}
r = requests.post('http://127.0.0.1:5000/item/table', data=data)
r.json()

In [ ]:
# items we want to store in our db
items_to_store = [
    ('PC', 200),
    ('TV', 500),
    ('pen', 1),
    ('sofa', 50)
]

In [ ]:
# store the items with api calls
for item in items_to_store:
    name = item[0]
    price = item[1]
    r = requests.post(f'http://127.0.0.1:5000/item/{name}', data={'price': price})
    print(r.json())

In [ ]:
# get sofa from API
r = requests.get(f'http://127.0.0.1:5000/item/sofa')
r.json()

In [ ]:
# update sofa with API
r = requests.put(f'http://127.0.0.1:5000/item/sofa', data={'price': 60})
r.json()

In [ ]:
# get sofa from API
r = requests.get(f'http://127.0.0.1:5000/item/sofa')
r.json()

In [ ]:
# get sofa from API
r = requests.delete(f'http://127.0.0.1:5000/item/sofa')
r.json()

In [ ]:
# get sofa from API
r = requests.get(f'http://127.0.0.1:5000/item/sofa')
r.json()

In [ ]:
# check the items
r = requests.get('http://127.0.0.1:5000/items')
data = r.json()

In [ ]:
df = pd.DataFrame(data['items'])
df

In [ ]:
df['price'].mean()